In [12]:
# ----------
# Background
# 
# A robotics company named Trax has created a line of small self-driving robots 
# designed to autonomously traverse desert environments in search of undiscovered
# water deposits.
#
# A Traxbot looks like a small tank. Each one is about half a meter long and drives
# on two continuous metal tracks. In order to maneuver itself, a Traxbot can do one
# of two things: it can drive in a straight line or it can turn. So to make a 
# right turn, A Traxbot will drive forward, stop, turn 90 degrees, then continue
# driving straight.
#
# This series of questions involves the recovery of a rogue Traxbot. This bot has 
# gotten lost somewhere in the desert and is now stuck driving in an almost-circle: it has
# been repeatedly driving forward by some step size, stopping, turning a certain 
# amount, and repeating this process... Luckily, the Traxbot is still sending all
# of its sensor data back to headquarters.
#
# In this project, we will start with a simple version of this problem and 
# gradually add complexity. By the end, you will have a fully articulated
# plan for recovering the lost Traxbot.
# 
# ----------
# Part One
#
# Let's start by thinking about circular motion (well, really it's polygon motion
# that is close to circular motion). Assume that Traxbot lives on 
# an (x, y) coordinate plane and (for now) is sending you PERFECTLY ACCURATE sensor 
# measurements. 
#
# With a few measurements you should be able to figure out the step size and the 
# turning angle that Traxbot is moving with.
# With these two pieces of information, you should be able to 
# write a function that can predict Traxbot's next location.
#
# You can use the robot class that is already written to make your life easier. 
# You should re-familiarize yourself with this class, since some of the details
# have changed. 
#
# ----------
# YOUR JOB
#
# Complete the estimate_next_pos function. You will probably want to use
# the OTHER variable to keep track of information about the runaway robot.
#
# ----------
# GRADING
# 
# We will make repeated calls to your estimate_next_pos function. After
# each call, we will compare your estimated position to the robot's true
# position. As soon as you are within 0.01 stepsizes of the true position,
# you will be marked correct and we will tell you how many steps it took
# before your function successfully located the target bot.

# These import steps give you access to libraries which you may (or may
# not) want to use.

from robot import *
from math import *
#from matrix import *
import random

# This is the function you have to write. The argument 'measurement' is a 
# single (x, y) point. This function will have to be called multiple
# times before you have enough information to accurately predict the
# next position. The OTHER variable that your function returns will be 
# passed back to your function the next time it is called. You can use
# this to keep track of important information over time.
def estimate_next_pos(measurement, OTHER = None):
    """Estimate the next (x, y) position of the wandering Traxbot
    based on noisy (x, y) measurements."""
    if OTHER is None:
        xy_estimate = measurement
        angle = 0
        num_measurements = 0
        total_step = 0
        total_angle_step = 0

        
    else:
        prev_num_measurements = OTHER[0]
        prev_measurement = OTHER[1]
        prev_angle = OTHER[2]
        total_step = OTHER[3]
        total_angle_step = OTHER[4]

        num_measurements = prev_num_measurements + 1

        step_size = distance_between(measurement, prev_measurement)
        total_step += step_size
        average_step = total_step / num_measurements


        angle = atan2(measurement[1]-prev_measurement[1],measurement[0]-prev_measurement[0])
        angle_step = angle - prev_angle
        #print "Angle step", angle_step, (angle_step%(2*pi)), angle_trunc(angle_step)
        
        total_angle_step += angle_trunc(angle_step)
        #total_angle_step += abs(angle_step) / 2
        average_angle_step = total_angle_step / num_measurements

        #new_angle = angle * 2 - prev_angle
        new_angle = angle + average_angle_step
        #print "Average angle step", average_angle_step

        x_estimate = measurement[0]+ average_step*cos(new_angle)
        y_estimate = measurement[1]+ average_step*sin(new_angle)
        #x_estimate = measurement[0]+ step_size*cos(new_angle)
        #y_estimate = measurement[1]+ step_size*sin(new_angle)
        xy_estimate = (x_estimate,y_estimate)


        
    OTHER = [num_measurements, measurement, angle, total_step, total_angle_step]
    #print "OTHER", OTHER
    # You must return xy_estimate (x, y), and OTHER (even if it is None) 
    # in this order for grading purposes.
    return xy_estimate, OTHER 

# A helper function you may find useful.
def distance_between(point1, point2):
    """Computes distance between point1 and point2. Points are (x, y) pairs."""
    x1, y1 = point1
    x2, y2 = point2
    return sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# This is here to give you a sense for how we will be running and grading
# your code. Note that the OTHER variable allows you to store any 
# information that you want. 
def demo_grading(estimate_next_pos_fcn, target_bot, OTHER = None):
    localized = False
    distance_tolerance = 0.01 * target_bot.distance
    ctr = 0
    # if you haven't localized the target bot, make a guess about the next
    # position, then we move the bot and compare your guess to the true
    # next position. When you are close enough, we stop checking.
    #For Visualization
    import turtle    #You need to run this locally to use the turtle module
    window = turtle.Screen()
    window.bgcolor('white')
    size_multiplier= 10  #change Size of animation
    broken_robot = turtle.Turtle()
    broken_robot.shape('turtle')
    broken_robot.color('green')
    broken_robot.resizemode('user')
    broken_robot.shapesize(0.1, 0.1, 0.1)
    measured_broken_robot = turtle.Turtle()
    measured_broken_robot.shape('circle')
    measured_broken_robot.color('red')
    measured_broken_robot.resizemode('user')
    measured_broken_robot.shapesize(0.1, 0.1, 0.1)
    prediction = turtle.Turtle()
    prediction.shape('arrow')
    prediction.color('blue')
    prediction.resizemode('user')
    prediction.shapesize(0.1, 0.1, 0.1)
    prediction.penup()
    broken_robot.penup()
    measured_broken_robot.penup()
    #End of Visualization
    while not localized and ctr <= 1000:
        ctr += 1
        measurement = target_bot.sense()
        position_guess, OTHER = estimate_next_pos_fcn(measurement, OTHER)
        print(position_guess)
        target_bot.move_in_circle()
        true_position = (target_bot.x, target_bot.y)
        print(true_position)
        print("")
        error = distance_between(position_guess, true_position)
        if error <= distance_tolerance:
            print("You got it right! It took you ", ctr, " steps to localize.")
            localized = True
        if ctr == 1000:
            print("Sorry, it took you too many steps to localize the target.")
        #More Visualization
        measured_broken_robot.setheading(target_bot.heading*180/pi)
        measured_broken_robot.goto(measurement[0]*size_multiplier, measurement[1]*size_multiplier-200)
        measured_broken_robot.stamp()
        broken_robot.setheading(target_bot.heading*180/pi)
        broken_robot.goto(target_bot.x*size_multiplier, target_bot.y*size_multiplier-200)
        broken_robot.stamp()
        prediction.setheading(target_bot.heading*180/pi)
        prediction.goto(position_guess[0]*size_multiplier, position_guess[1]*size_multiplier-200)
        prediction.stamp()
        #End of Visualization
    window.bye()
    return localized

# This is a demo for what a strategy could look like. This one isn't very good.
def naive_next_pos(measurement, OTHER = None):
    """This strategy records the first reported position of the target and
    assumes that eventually the target bot will eventually return to that 
    position, so it always guesses that the first position will be the next."""
    if not OTHER: # this is the first measurement
        OTHER = measurement
    xy_estimate = OTHER 
    return xy_estimate, OTHER

# This is how we create a target bot. Check the robot.py file to understand
# How the robot class behaves.
test_target = robot(2.1, 4.3,0.5, 2*pi / 34.0, 1.5)
measurement_noise = 0.05*test_target.distance
test_target.set_noise(0.0, 0.0, measurement_noise)

demo_grading(estimate_next_pos, test_target)
#position_guess,OTHER = estimate_next_pos(test_target.sense(), OTHER=None)
#print(test_target.sense())
#print(position_guess)

(2.096628279932661, 4.252296076756509)
(3.2618187593136714, 5.248776670511476)

(3.679161045000088, 6.725537844524444)
(4.229518090705041, 6.394882252046046)

(4.572840166324683, 7.832056086532258)
(4.970144154070997, 7.699287493626686)

(5.146086209394509, 9.460007116411177)
(5.4584758171066685, 9.117572439213015)

(5.791235303907695, 10.571881822777428)
(5.677883530716649, 10.601439096028392)

(5.201627398704583, 12.07274262198747)
(5.620895628364498, 12.10035616477686)

(5.157789458000387, 13.602854858578679)
(5.289452764715364, 13.563279822435065)

(4.693385621920484, 14.832194784103542)
(4.694841828968918, 14.940391958424991)

(3.5886095415369788, 15.967868497401154)
(3.857311583384793, 16.18479667061946)

(2.828742338712478, 17.26573681634073)
(2.8053831159695455, 17.25411724904363)

(1.4317859102690669, 18.034376793879634)
(1.574878589031866, 18.11193926390795)

(0.22441987810176323, 18.5626224651304)
(0.20770135840910497, 18.729050615340313)

(-1.3554018255555687, 18.9405655036

(2.051543836304795, 4.379721231450022)
(2.0999999999996635, 4.300000000000042)

(3.229460984101542, 5.229781135925372)
(3.2618187593133454, 5.248776670511504)

(4.188328862220083, 6.504076568574752)
(4.229518090704728, 6.394882252046065)

(5.050358575998691, 7.746906944848403)
(4.970144154070698, 7.699287493626697)

(5.422054303052796, 9.231469998921089)
(5.458475817106384, 9.11757243921302)

(5.41198703717643, 10.723701217153165)
(5.677883530716381, 10.601439096028393)

(5.811349410361461, 12.137222385307568)
(5.620895628364246, 12.100356164776862)

(5.452548702066419, 13.632351102208977)
(5.2894527647151275, 13.56327982243507)

(4.448218125322726, 14.588085380913492)
(4.694841828968697, 14.940391958425003)

(4.133054228119668, 16.28175740611506)
(3.8573115833845852, 16.18479667061948)

(2.8589796442634356, 17.116715158267837)
(2.805383115969349, 17.254117249043667)

(1.7882066550405917, 18.385532528011908)
(1.5748785890316788, 18.111939263907995)

(0.09616726512671048, 18.37841152513

True